<a href="https://colab.research.google.com/github/Rafilksz/Pyspark-Criascript/blob/master/Pyspark%20Criascript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=91b90d0d7fb8379541475c1200fd9d2cfa7fe0300191e558997c3c374bf670e4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


#salve meu bom, refiz este notebook organizando tudo em um codigo só, sem separar por linhas, nao me critique
#vou comentar a maioria do codigo para ficar clara minha linha de raciocinio kkkkkk se algo estiver errado avise

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, FloatType
from pyspark.sql.functions import col, avg, min, max, from_unixtime, expr


spark = SparkSession.builder.getOrCreate()#crio a seção

arquivo = "/content/drive/MyDrive/spotify.csv"#indico caminho


In [ ]:
# Definir o esquema de cada coluna da maneira que eu quero
schema = StructType([
    StructField("Unnamed: 0", IntegerType(), True),
    StructField("track_id", StringType(), True),
    StructField("artists", StringType(), True),
    StructField("album_name", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("popularity", FloatType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("explicit", BooleanType(), True),
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("track_genre", StringType(), True),
])

# Ler o arquivo CSV usando o esquema da forma que eu setei acima
df = spark.read.format('csv') \
    .option('header', 'True') \
    .schema(schema) \
    .csv(arquivo)


df.printSchema()#pedi o esquema para eu ver se ta da forma que eu setei

root
 |-- Unnamed: 0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: float (nullable = true)
 |-- track_genre: string (nullable = true)



In [ ]:
type(df)#confirmo se ta em pyspark, só por curiosidade

pyspark.sql.dataframe.DataFrame

In [ ]:
df.take(5)#pego uma amostra de 5 para ver como fica

[Row(Unnamed: 0=0, track_id='5SuOikwiRyPMVoIQDJUgSV', artists='Gen Hoshino', album_name='Comedy', track_name='Comedy', popularity=73.0, duration_ms=230666, explicit=False, danceability=0.6759999990463257, energy=0.460999995470047, key=1, loudness=-6.745999813079834, mode=0, speechiness=0.14300000667572021, acousticness=0.03220000118017197, instrumentalness=1.0100000054080738e-06, liveness=0.3580000102519989, valence=0.7149999737739563, tempo=87.91699981689453, time_signature=4.0, track_genre='acoustic'),
 Row(Unnamed: 0=1, track_id='4qPNDBW1i3p13qLCt0Ki3A', artists='Ben Woodward', album_name='Ghost (Acoustic)', track_name='Ghost - Acoustic', popularity=55.0, duration_ms=149610, explicit=False, danceability=0.41999998688697815, energy=0.16599999368190765, key=1, loudness=-17.235000610351562, mode=1, speechiness=0.0763000026345253, acousticness=0.9240000247955322, instrumentalness=5.559999863180565e-06, liveness=0.10100000351667404, valence=0.2669999897480011, tempo=77.48899841308594, ti

In [ ]:
display(df.show(5))#aqui uso a função display para organizar esta merda e ficar mais bonito de ler

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|      73.0|     230666|   false|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|

None

In [ ]:
df.count()#conto quantas linhas tem esta bagaça

114000

In [ ]:
df.filter("popularity >= 95").show(100)#filtro as que tem popularidade acima ou igual a 95 e printei todas até o limite de 100

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|     20000|5IgjP7X4th6nMNDh4...|         Chris Brown|   Indigo (Extended)| Under The Influence|      96.0|     184613|    true|       0.733|  0.69|  9|  -5.529|   0|     0.0427|      0.0635|         1.18E-6|   0.105|

In [ ]:
#printei todos os estilos para escolher qual eu iria trabalhar
#col é para referenciar colunas em pyspike
genres = df.select("track_genre").distinct().orderBy(col("track_genre").asc()).collect()
for genre_row in genres:
    print(genre_row["track_genre"])


-12.753
0.0898
0.103
0.114
0.159
0.34
0.576
0.983
1
1.5e-05
10
105.188
114.211
117.11
121.165
125.262
131.721
133.106
133.67
134.113
148.759
151.539
3
4
5
60.015
68.453
68.958
7
74.077
76.691
76.791
81.078
89.01
89.912
90.051
91.467
95.073
acoustic
afrobeat
alt-rock
alternative
ambient
anime
black-metal
bluegrass
blues
brazil
breakbeat
british
cantopop
chicago-house
children
chill
classical
club
comedy
country
dance
dancehall
death-metal
deep-house
detroit-techno
disco
disney
drum-and-bass
dub
dubstep
edm
electro
electronic
emo
folk
forro
french
funk
garage
german
gospel
goth
grindcore
groove
grunge
guitar
happy
hard-rock
hardcore
hardstyle
heavy-metal
hip-hop
honky-tonk
house
idm
indian
indie
indie-pop
industrial
iranian
j-dance
j-idol
j-pop
j-rock
jazz
k-pop
kids
latin
latino
malay
mandopop
metal
metalcore
minimal-techno
mpb
new-age
opera
pagode
party
piano
pop
pop-film
power-pop
progressive-house
psych-rock
punk
punk-rock
r-n-b
reggae
reggaeton
rock
rock-n-roll
rockabilly
romance
sa

In [ ]:
#aqui vou pegar preferencia de musicas do aki ta gay, ja que ele disse que gosta de musica de anime, e existeo genero na lista

df_anime = df.filter(df["track_genre"] == "anime")#busco pelo genero anime
df_anime_top10 = df_anime.orderBy(df_anime["popularity"].desc()).limit(10)#ordeno por popularidade
df_anime_top10.show(truncate=False)#mostro as 10 e dou truncate para ficar facil de ler e organizado

saida_anime = "/content/anime.csv" #nome do arquivo de anime que tratei
df_anime_top10.write.csv(saida_anime, header=True, mode="overwrite")#salvei como csv para usar power bi



+----------+----------------------+----------------+--------------------------+-----------------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|track_id              |artists         |album_name                |track_name                   |popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |time_signature|track_genre|
+----------+----------------------+----------------+--------------------------+-----------------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|5002      |3khEEPRyBeOUabbmOPJzAG|Kenshi Yonezu   |KICK BACK                 |KICK BACK                    |83.0      |193495     |false   |0.577       |0.941 |1  |-5.17   |1   

In [ ]:
#o mesmo acima para o cria, ja que ele gosta de musica "skate" vou meter um charlie marrom juninho

df_charlie_brown = df.filter(df["artists"].like(f"%Charlie Brown Jr.%"))# Buscar músicas do artista "Charlie Brown Jr."
df_charlie_brown_unique = df_charlie_brown.dropDuplicates(["track_name"]) \
    .orderBy(col("popularity").desc()) \
    .limit(10)# Remover duplicatas na coluna "track_name" e ordenar por popularidade
df_charlie_brown_unique.show(truncate=False)# Mostrar as 10 músicas únicas do artista "Charlie Brown Jr." ordenadas por popularidade

saida_skate = "/content/skate.csv" #nome do arquivo de skate que tratei
df_charlie_brown_unique.write.csv(saida_skate, header=True, mode="overwrite")#salvei como csv para usar power bi


+----------+----------------------+------------------------------+-----------------------------------------------+----------------------------------------------------------------------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|track_id              |artists                       |album_name                                     |track_name                                                                        |popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |time_signature|track_genre|
+----------+----------------------+------------------------------+-----------------------------------------------+----------------------------------------------------------------------------------+----------+-----------+--------+------------+------+---+--------+----+---------

In [ ]:
df_popular = df.filter("popularity >= 95")# Filtrar por popularidade acima ou igual a 95
df_popular_unique_sorted = df_popular.dropDuplicates(["track_name"]) \
    .orderBy(col("popularity").cast("double").desc())# Remover duplicatas na coluna "track_name" e ordenar por popularidade em ordem decrescente
df_popular_unique_sorted.show(100, truncate=False)# Mostrar as músicas únicas com popularidade acima de 95, ordenadas por popularidade em ordem decrescente

saida_pop100_95 = "/content/pop95mais.csv" #nome do arquivo de populares nota acima 95 que tratei
df_popular_unique_sorted.write.csv(saida_pop100_95, header=True, mode="overwrite")#salvei como csv para usar power bi


+----------+----------------------+--------------------------+-----------------------------------------------------------------------+-------------------------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|track_id              |artists                   |album_name                                                             |track_name                           |popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |time_signature|track_genre|
+----------+----------------------+--------------------------+-----------------------------------------------------------------------+-------------------------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--

In [ ]:
# Calcular estatísticas de danceability
danceability_stats = df.select(avg("danceability").alias("avg_danceability"),
                               min("danceability").alias("min_danceability"),
                               max("danceability").alias("max_danceability")).first()

# Média, Mínimo e Máximo de Danceability
avg_danceability = danceability_stats["avg_danceability"]
min_danceability = danceability_stats["min_danceability"]
max_danceability = danceability_stats["max_danceability"]

# Mostrar as estatísticas
print("Média de Danceability:", avg_danceability)
print("Danceability Mínimo:", min_danceability)
print("Danceability Máximo:", max_danceability)

# Exibir detalhes das músicas correspondentes ao mínimo e máximo de danceability
print("\nDetalhes da Música com Danceability Mínimo:")
df_min_danceability = df.filter(col("danceability") == min_danceability).limit(1)
df_min_danceability.select("track_name", "artists", "danceability").show(truncate=False)

print("\nDetalhes da Música com Danceability Máximo:")
df_max_danceability = df.filter(col("danceability") == max_danceability).limit(1)
df_max_danceability.select("track_name", "artists", "danceability").show(truncate=False)


# Especificar o caminho onde você deseja salvar o arquivo CSV para as estatísticas de danceability
caminho_arquivo_danceability_stats = "/content/danceability_stats.csv"

# Criar DataFrame para as estatísticas de danceability
df_danceability_stats = spark.createDataFrame([(avg_danceability, min_danceability, max_danceability)],
                                              ["avg_danceability", "min_danceability", "max_danceability"])

# Salvar o DataFrame em formato CSV
df_danceability_stats.write.csv(caminho_arquivo_danceability_stats, header=True, mode="overwrite")

# Especificar o caminho onde você deseja salvar os arquivos CSV para as músicas com danceability mínimo e máximo
caminho_arquivo_min_danceability = "/content/min_danceability.csv"
caminho_arquivo_max_danceability = "/content//max_danceability.csv"

# Salvar os DataFrames em formato CSV
df_min_danceability.write.csv(caminho_arquivo_min_danceability, header=True, mode="overwrite")
df_max_danceability.write.csv(caminho_arquivo_max_danceability, header=True, mode="overwrite")



Média de Danceability: 31.79821658843467
Danceability Mínimo: 0.0
Danceability Máximo: 549809.0

Detalhes da Música com Danceability Mínimo:
+-------------+---------------------+------------+
|track_name   |artists              |danceability|
+-------------+---------------------+------------+
|The Departure|Max Richter;Lang Lang|0.0         |
+-------------+---------------------+------------+


Detalhes da Música com Danceability Máximo:
+--------------+-----------+------------+
|track_name    |artists    |danceability|
+--------------+-----------+------------+
| Live in Odeon|Estas Tonne|549809.0    |
+--------------+-----------+------------+



In [ ]:
# Calcular estatísticas de loudness
loudness_stats = df.select(avg("loudness").alias("avg_loudness"),
                           min("loudness").alias("min_loudness"),
                           max("loudness").alias("max_loudness")).first()

# Média, Mínimo e Máximo de Loudness
avg_loudness = loudness_stats["avg_loudness"]
min_loudness = loudness_stats["min_loudness"]
max_loudness = loudness_stats["max_loudness"]

# Mostrar as estatísticas
print("Média de Loudness:", avg_loudness)
print("Loudness Mínimo:", min_loudness)
print("Loudness Máximo:", max_loudness)

# Exibir detalhes das músicas correspondentes  mínimo e máximo de loudness
print("\nDetalhes da Música com Loudness Mínimo:")
df_min_loudness = df.filter(col("loudness") == min_loudness).limit(1)
df_min_loudness.select("track_name", "artists", "loudness").show(truncate=False)

print("\nDetalhes da Música com Loudness Máximo:")
df_max_loudness = df.filter(col("loudness") == max_loudness).limit(1)
df_max_loudness.select("track_name", "artists", "loudness").show(truncate=False)



# Especificar o caminho onde você deseja salvar o arquivo CSV para as estatísticas de loudness
caminho_arquivo_loudness_stats = "/content/loudness_stats.csv"

# Criar DataFrame para as estatísticas de loudness
df_loudness_stats = spark.createDataFrame([(avg_loudness, min_loudness, max_loudness)],
                                          ["avg_loudness", "min_loudness", "max_loudness"])

# Salvar o DataFrame em formato CSV
df_loudness_stats.write.csv(caminho_arquivo_loudness_stats, header=True, mode="overwrite")


# Especificar o caminho onde você deseja salvar os arquivos CSV para as músicas com loudness mínimo e máximo
caminho_arquivo_min_loudness = "/content/min_loudness.csv"
caminho_arquivo_max_loudness = "/content/max_loudness.csv"

# Salvar os DataFrames em formato CSV
df_min_loudness.write.csv(caminho_arquivo_min_loudness, header=True, mode="overwrite")
df_max_loudness.write.csv(caminho_arquivo_max_loudness, header=True, mode="overwrite")




Média de Loudness: 0.051286616845378837
Loudness Mínimo: -49.53099822998047
Loudness Máximo: 471050.0

Detalhes da Música com Loudness Mínimo:
+--------------------------------------------+------------------------+--------+
|track_name                                  |artists                 |loudness|
+--------------------------------------------+------------------------+--------+
|Soothing White Noise - Loopable With No Fade|White Noise Sleep Sounds|-49.531 |
+--------------------------------------------+------------------------+--------+


Detalhes da Música com Loudness Máximo:
+-----------+-------------------+--------+
|track_name |artists            |loudness|
+-----------+-------------------+--------+
| Snuff Crew|Robert Owens;Atjazz|471050.0|
+-----------+-------------------+--------+



In [ ]:
df.write.csv('/content/minha_tarefa_dataframe.csv', header=True, mode='overwrite')#salvo a porra toda

Terminei, agora vou FAZER O DASHBOARD DISSO

apos concluir, ja posso falar que tenho dado em casa?
